<a href="https://colab.research.google.com/github/TernFolbaek/angryGPT/blob/googleColab/angryGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/TernFolbaek/angryGPT.git

Cloning into 'angryGPT'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 25 (delta 3), reused 24 (delta 2), pack-reused 0
Unpacking objects: 100% (25/25), 6.94 MiB | 4.92 MiB/s, done.
https://git-lfs.github.com/spec/v1 is not a valid attribute name: .gitattributes:1
sha256:0b8038351cc3dc9e97ff08497edaba9a1e699c0ea4fa190093b15de3e632cb10 is not a valid attribute name: .gitattributes:2


In [2]:
!pip install pandas scikit-learn matplotlib convokit transformers 
!ls
!git lfs install

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.0/168.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.2 MB/s eta 0:00:00
  Created wheel for convokit: filename=convokit-2.5.3-py3-none-any.whl size=204125 sha256=1d8b40a779d1e9259e4b715124f1901ed6ff882cb891cd71a6f1d433c3b13527
  Stored in directory

In [3]:
%cd angryGPT
!pwd

/content/angryGPT
/content/angryGPT


In [ ]:
!git remote show origin
!git lfs pull

* remote origin
  Fetch URL: https://github.com/TernFolbaek/angryGPT.git
  Push  URL: https://github.com/TernFolbaek/angryGPT.git
  HEAD branch: main
  Remote branch:
    main tracked
  Local branch configured for 'git pull':
    main merges with remote main
  Local ref configured for 'git push':
    main pushes to main (up to date)


In [ ]:
with open('utterances.jsonl', 'r') as file:
    for i, line in enumerate(file):
        print(line)
        if i >= 5:
            break


{"id": "cue8y0b", "conversation_id": "cue8y0b", "text": "(Okay, I've seen this view come up a few times before and I've always been unsuccessful in convincing people about why they're wrong. However, it seems that your view is based on studies, so maybe you'll respond well to evidence /u/Carlosriccy.)\n\nI cannot dispute the fact that there is a measurable IQ gap b/w white students and black students in the US. However, it is most certainly not because of genetics. Almost all of it can be attributed to social causes - black students are often in schools that aren't adequately funded, poverty and its corollary effects, etc. \n\nThe genetic basis for difference in IQ has been tested. You can find a comprehensive review [here](http://www.ncbi.nlm.nih.gov/pmc/articles/PMC3341646/). If you look at the science it's clear that there is no evidence that justifies making inferences about race and intelligence.\n\nBut what about all those studies in the past that showed the evidence?! Well, it's

In [ ]:
!pip install huggingface_hub
!huggingface-cli login

import pandas as pd
import sklearn
import string
import numpy as np
import matplotlib.pyplot as plt
import pickle
from convokit import Corpus, download
from pandas import json_normalize
import json
import os
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import (GPT2LMHeadModel, GPT2Tokenizer, GPT2Config,
                          DataCollatorForLanguageModeling, Trainer, TrainingArguments)

# Load the corpus
corpus = Corpus(filename=download("conversations-gone-awry-corpus"))
corpus.print_summary_stats()

# Load JSON data and convert to DataFrame
jsonl_file = 'utterances.jsonl'
with open('utterances.jsonl', 'r') as file:
    data = [json.loads(line) for line in file]
flat_data = json_normalize(data)
df = pd.DataFrame(flat_data)
columns_to_drop = [column for column in df.columns if column != 'text']
df = df.drop(columns=columns_to_drop)

flat_data = json_normalize(data)
df = pd.DataFrame(flat_data)
columns_to_drop = [column for column in df.columns if column != 'text']
df = df.drop(columns=columns_to_drop)

# Save DataFrame to CSV file
csv_file = 'provoking_file.csv'
df.to_csv(csv_file, index=False)

# Function to remove punctuation and numbers from text
def remove_pun_and_numbers(text):
    text = str(text)
    to_remove = string.punctuation + string.digits
    text = text.lower()
    return text.translate(str.maketrans('', '', to_remove))

# Load model
def load_model():
    with open('model.pkl', 'rb') as file:
        model = pickle.load(file)
    return model

model = load_model()

# Add the new column to the DataFrame
def only_negatives(message):
    result = model.predict([message])
    if result == ['Negative']:
        return message
    else:
        return None

df['text'] = df['text'].apply(only_negatives)
df = df.dropna(subset=['text'])
df = df.reset_index(drop=True)

# Split the dataset
split_index = len(df) // 2
output_df = df.iloc[split_index:].copy()
input_df = df.iloc[:split_index].copy()
output_df.rename(columns={'text': 'output'}, inplace=True)
input_df.rename(columns={'text': 'input'}, inplace=True)
output_df.reset_index(drop=True, inplace=True)
input_df.reset_index(drop=True, inplace=True)
df = pd.concat([input_df, output_df], axis=1)
df.output.apply(remove_pun_and_numbers)
df.input.apply(remove_pun_and_numbers)

from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")



# Load pre-trained model and tokenizer
def from_local_pretrained(directory):
    config = GPT2Config.from_pretrained(os.path.join(directory, "config.json"))
    tokenizer = GPT2Tokenizer.from_pretrained(directory)
    model = GPT2LMHeadModel.from_pretrained(os.path.join(directory, "pytorch_model.bin"), config=config)
    return tokenizer, model

local_distilgpt2_directory = "distilgpt2"
tokenizer, model = from_local_pretrained(local_distilgpt2_directory)

# Prepare the dataset
eos_token = tokenizer.eos_token
df["text"] = df["input"] + eos_token + df["output"]
df["text"].to_csv("processed_dataset.txt", index=False, header=False)

class CustomTextDataset(Dataset):
    def __init__(self, tokenizer, file_path, block_size):
        with open(file_path, "r") as f:
            text = f.read()
        
        self.tokenizer = tokenizer
        self.block_size = block_size
        self.examples = tokenizer(text, return_tensors="pt", truncation=True, padding=False)["input_ids"]
    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx]

# Create a CustomTextDataset and DataCollator
dataset = CustomTextDataset(tokenizer=tokenizer, file_path="processed_dataset.txt", block_size=128)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Set up training arguments
training_args = TrainingArguments(
    output_dir="chatbot_output",
    overwrite_output_dir=True,
    num_train_epochs= 10,
    per_device_train_batch_size=1,
    save_steps=10_000,
    gradient_accumulation_steps=1,
    save_total_limit=2,
    logging_steps=500,
)

# Train the model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

trainer.train()

model.save_pretrained("my_model")
model = GPT2LMHeadModel.from_pretrained("my_model")

   


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful
Number of Speakers: 8069
Number of Utterances: 30021
Number of Conversations: 418

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


In [ ]:
input_text = ["race intelligence"]
input_ids = tokenizer(input_text, return_tensors='pt').input_ids
generated_text = model.generate(input_ids, max_length=100, num_return_sequences=1, do_sample=True)
decoded_text = tokenizer.decode(generated_text[0], skip_special_tokens=True)
print(decoded_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


race intelligence and the effects of the energy. So you get a little more complex stuff when you try to be the person who makes up the science and the social analysis - so you get an interesting framework for this. Then you get a little more sophisticated stuff when you're looking at things like that. This is a really good framework and you can find what really sets the world apart. I think this has a very powerful effect on your thinking and is a good way to get the research needed because I
